### カリキュラム - 【AIコース】CNNによる画像分類 実装2

##### ・前処理

In [1]:
import keras
# mnistデータインポート
from keras.datasets import mnist
# Sequentialモデルのインポート
from keras.models import Sequential
# 通常の全結合ニューラルネットワークレイヤDenseのインポート
from keras.layers import Dense
# 一次元配列に変換するFlattenのインポート
from keras.layers import Flatten
# 二次元入力をフィルタする畳み込み層Conv2Dのインポート
from keras.layers import Conv2D
# 二次元空間データのマックスプーリング演算MaxPooling2Dのインポート
from keras.layers import MaxPooling2D

Using TensorFlow backend.
/Users/yasu/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data() 

In [3]:
print(X_train.shape)

(60000, 28, 28)


In [4]:
print(y_train.shape)

(60000,)


In [5]:
print(X_test.shape)

(10000, 28, 28)


In [6]:
print(y_test[0:10])

[7 2 1 0 4 1 4 9 5 9]


In [7]:
# 画像サイズの取得
img_rows = X_train[0].shape[1]
img_cols = X_train[0].shape[0]
print("img_rows:", img_rows, "/img_cols:", img_cols)

img_rows: 28 /img_cols: 28


In [8]:
# 何種類に分類しているかを確認するために正解ラベルをSET型に変換
num_classes = len(set(y_train))
num_classes

10

In [9]:
# チャネルを表すために一次元追加
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
print("X_train.shape:", X_train.shape, "/X_test.shape:", X_test.shape)

X_train.shape: (60000, 28, 28, 1) /X_test.shape: (10000, 28, 28, 1)


In [10]:
input_shape = X_train.shape[1:4]
input_shape

(28, 28, 1)

In [11]:
# 色情報を０〜１の範囲に正規化
X_train = X_train.astype("float32")  ## astypeはndarryの要素のデータ型を別のデータ型にしたndarrayを生成
X_test = X_test.astype("float32")
X_train /= 255  ## X_train = X_train / 255
X_test /= 255

In [12]:
# クラスベクトルをバイナリのクラス行列に変換
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

##### ・モデル設計

In [13]:
model = Sequential()
# 畳み込み層１
model.add(Conv2D(2, # チューニング２ - 1から2に変更
                 kernel_size = (3, 3),
                 strides = (1, 1),
                 padding = "same",
                 activation = "relu",
                 bias = True,
                 input_shape = input_shape))
# プーリング層１
from keras.layers import AveragePooling2D
model.add(AveragePooling2D(pool_size = (2, 2), # チューニング１６ - MaxPooling2DからAveragePooling2Dへ
                       strides = None,
                       padding = "same")) # チューニング１８ - validからsameに変更
# 畳み込み層２
model.add(Conv2D(2, # チューニング２ - 1から2に変更
                 kernel_size = (3, 3),
                 strides = (1, 1),
                 padding = "same",
                 bias = True,
                 activation = "relu"))
# プーリング層２
model.add(AveragePooling2D(pool_size = (2, 2),  # チューニング１６ - MaxPooling2DからAveragePooling2Dへ
                       strides = None,
                       padding = "same")) # チューニング１８ - validからsameに変更
model.add(Flatten())
# 全結合層１
model.add(Dense(256, # チューニング３- 128から256へ
                activation = "relu"))
# チューニング４ - 全結合層２の追加
model.add(Dense(256, # チューニング３- 128から256へ
                activation = "relu"))
# 全結合層 出力
model.add(Dense(num_classes,
                activation = "softmax"))
# モデルのコンパイル
model.compile(loss = keras.losses.categorical_crossentropy,
              optimizer = keras.optimizers.Adadelta(),
              metrics = ["accuracy"])

/Users/yasu/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/ipykernel/__main__.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(2, kernel_size=(3, 3), strides=(1, 1), padding="same", activation="relu", input_shape=(28, 28, 1..., use_bias=True)`
/Users/yasu/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/ipykernel/__main__.py:21: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(2, kernel_size=(3, 3), strides=(1, 1), padding="same", activation="relu", use_bias=True)`


##### ・実行

In [14]:
model.fit(X_train, y_train,
          batch_size = 256, # チューニング１１- 128から256へ
          epochs = 4, # チューニング１２- 3から4へ
          verbose = 1,
          validation_data = (X_test, y_test))
score = model.evaluate(X_test, y_test, verbose = 1)  ## バッチごとのロスの計算
print("Test loss: ", score[0])
print("Test accuracy: ", score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/4
60000/60000 [==============================] - 10s 164us/step - loss: 0.6235 - acc: 0.8172 - val_loss: 0.2650 - val_acc: 0.9183

##### ・チューニングポイントと結果
*** チューニング１９（mix）: チューニング２ & チューニング３ & チューニング４ & チューニング１１  & チューニング１２ & チューニング１６ **  
結果: 改善 - 達成  
Test loss:  0.0995357605973  
Test accuracy:  0.9685  

*** チューニング１８: AveragePooling2Dのpaddingをvalidからsameに変更 & チューニング３ & チューニング４ & チューニング１１  & チューニング１２ & チューニング１６ **  
ポイント: model.add(AveragePooling2D(pool_size = (2, 2),... padding = "same"))   
結果: 改善 - 達成  
Test loss:  0.118778183427  
Test accuracy:  0.9629  

*** チューニング１７: 畳み込み層とAveragePooling2Dのプーリング層を一つずつ追加 & チューニング３ & チューニング４ & チューニング１１  & チューニング１２ & チューニング１６ **   
ポイント:  
model.add(AveragePooling2D(pool_size = (2, 2),
                       strides = None,
                       padding = "same"))
model.add(Conv2D(2,  
                 kernel_size = (3, 3),
                 strides = (1, 1),
                 padding = "same",
                 bias = True,
                 activation = "relu"))  
結果: 悪化 - 大   
Test loss:  0.546606017733  
Test accuracy:  0.8147  

*** チューニング１６: 二つのプーリング層をMaxPooling2DからAveragePooling2Dに変更 & チューニング３ & チューニング４ & チューニング１１  & チューニング１２ **  
ポイント:  
from keras.layers import AveragePooling2D  
model.add(AveragePooling2D(...  
結果: 改善 - 達成  
Test loss:  0.132774380331  
Test accuracy:  0.9603  

*** チューニング１５（mix）: チューニング３ & チューニング４ & チューニング５ & チューニング１１  & チューニング１２ **  
結果: 悪化 - 最悪  
Test loss:  0.662738190937  
Test accuracy:  0.772  

*** チューニング１４（mix）: チューニング３ & チューニング４ & チューニング５ & チューニング１０ & チューニング１１ & チューニング１２ **  
結果: 悪化 - 大  
Test loss:  0.344120242673  
Test accuracy:  0.8895  

*** チューニング１３（mix）: チューニング３ & チューニング４ & チューニング１０ & チューニング１１ & チューニング１２ **  
結果: 悪化  
Test loss:  0.194750523516  
Test accuracy:  0.9358  

*** チューニング１２: epochsを３から４に変更 & チューニング３ & チューニング４ & チューニング１１ **  
ポイント: model.fit(X_train, y_train, ... epochs = 4,...  
結果: 悪化  
Test loss:  0.126296335971  
Test accuracy:  0.9581

*** チューニング１１: batch_sizeを256に変更 & チューニング３ & チューニング４ **  
ポイント: model.fit(X_train, y_train, batch_size = 256...  
結果: 改善 - 達成  
Test loss:  0.138864753012  
Test accuracy:  0.9549  

*** チューニング１０: 12.5%のDropoutの追加 & チューニング３ & チューニング４ **  
ポイント:  
from keras.layers import Dropout  
model.add(Dropout(0.25))  
結果: 改善 - 達成しかしチューニング８より低い    
Test loss:  0.157561711765
Test accuracy:  0.9509

*** チューニング９: チューニング３ & チューニング４ & チューニング６ **  
結果: 悪化  
Test loss:  0.187988321602  
Test accuracy:  0.9421  

*** チューニング８: チューニング３ & チューニング４ **  
結果: 改善 - 達成  
Test loss:  0.146716109802  
Test accuracy:  0.9521  

*** チューニング７: Batch Normalizationを二つ追加 & チューニング４ **  
ポイント:  
from keras.layers import BatchNormalization  
model.add(BatchNormalization())  
結果: 悪化 - 中
Test loss:  0.345551757592  
Test accuracy:  0.8936  

*** チューニング６: 25%のDropoutの追加 & チューニング４ **  
ポイント:  
from keras.layers import Dropout  
model.add(Dropout(0.5))  
結果: 悪化  
Test loss:  0.22159668749  
Test accuracy:  0.9312

*** チューニング５: 畳み込み層とプーリング層を一つずつ追加 & チューニング４**  
ポイント:  
model.add(Conv2D(1,
                 kernel_size = (3, 3),
                 strides = (1, 1),
                 padding = "same",
                 bias = True,
                 activation = "relu"))  
model.add(MaxPooling2D(pool_size = (2, 2),
                       strides = None,
                       padding = "valid"))  
結果: 悪化 - 大  
Test loss:  0.568839028692  
Test accuracy:  0.8106  

*** チューニング４: 全結合層を一つ追加 **  
ポイント: model.add(Dense(128, activation = "relu"))  
結果: 改善 - 未達成   
Test loss:  0.162400710883  
Test accuracy:  0.9481

*** チューニング３: Denseのノード数を１２８から２５６に変更 **  
ポイント: model.add(Dense(256,...  
結果: 悪化  
Test loss:  0.208994126648  
Test accuracy:  0.9329

*** チューニング２: Conv2Dのフィルタを１から２に変更 **  
ポイント: model.add(Conv2D(2,...    
結果: 悪化  
Test loss:  0.188836201447  
Test accuracy:  0.9418

*** チューニング１: デフォルト **  
Test loss:  0.190458923683  
Test accuracy:  0.9426  